In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

import kraft

In [ ]:
v = np.random.random_sample(size=(50, 50))

r = 5

## factorize_matrix_by_nmf

In [ ]:
w, h, error = kraft.matrix_factorization.factorize_matrix_by_nmf(v, r)

## factorize_matrix

## plot_matrix_factorization

In [ ]:
kraft.matrix_factorization.plot_matrix_factorization((w,), (h,))

## solve_ax_b

In [ ]:
for mode in ("ws", "hs"):

    ws, hs, errors = kraft.matrix_factorization.factorize_matrix((v,), r, mode)

    kraft.matrix_factorization.plot_matrix_factorization(ws, hs, errors=errors)

In [ ]:
for mode in ("ws", "hs"):

    ws, hs, errors = kraft.matrix_factorization.factorize_matrix((v, v * 10), r, mode)

    kraft.matrix_factorization.plot_matrix_factorization(ws, hs, errors=errors)